In [82]:
import os
import shutil
import numpy as np
import pickle as pk
import pandas as pd
from keras.utils import to_categorical ,Sequence
from keras import losses, models, optimizers
from keras.models import load_model
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import Activation, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from sklearn.model_selection import KFold
from random_eraser import get_random_eraser
from keras.optimizers import Adam
from os.path import join
import resnet
from sklearn.utils import shuffle

map_dict = pk.load(open('data/map.pkl' , 'rb'))


In [66]:
semi = pd.read_csv('data/cotrain/Y_selftrain_ens_verified.csv')

In [67]:
semi_map = {}

semi_name = semi['fname'].values
semi_label_verified = semi['label_verified'].values

for idx ,d in enumerate( semi_name):
    semi_map[d] = semi_label_verified[idx]


In [76]:
unverified_df = pd.read_csv('data/train_label.csv')
test_df = pd.read_csv('data/sample_submission.csv')

unverified_df = unverified_df[unverified_df['fname'].isin(semi_name)]
unverified_df = unverified_df.drop(columns=['manually_verified'])
unverified_df['label_verified'] = unverified_df['fname'].map(semi_map)

test_df = test_df[test_df['fname'].isin(semi_name)]
test_df['label_verified'] = test_df['fname'].map(semi_map)

unverified_idx = unverified_df.index.values
test_idx = test_df.index.values

df = pd.concat([unverified_df , test_df])
df = df.drop(columns=['label'])
df['trans'] = df['label_verified'].map(map_dict)
df['onehot'] = df['trans'].apply(lambda x: to_categorical(x,num_classes=41))

In [80]:
X_unverified = np.load('data/mfcc/X_train.npy')[unverified_idx]
X_test = np.load('data/X_test.npy')[test_idx]

X_semi = np.append(X_unverified,X_test , axis=0)
Y_semi = np.array(df['onehot'].tolist()).reshape(-1,41)

print(X_semi.shape)
print(Y_semi.shape)

(4823, 40, 345, 1)
(4823, 41)


In [81]:
 # data generator ====================================================================================
class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

# Training Semi Data

In [83]:

model_path = 'model_full_resnet2'
refine_path = 'model_full_resnet2_refine_co'

all_x = np.concatenate( (np.load('data/mfcc/X_train.npy') , np.load('data/X_test.npy')))

if not os.path.exists(refine_path):
    os.mkdir(refine_path)

for i in range(1,11):
    X_train = np.load('data/ten_fold_data/X_train_{}.npy'.format(i)) 
    Y_train = np.load('data/ten_fold_data/Y_train_{}.npy'.format(i)) 
    X_test = np.load('data/ten_fold_data/X_valid_{}.npy'.format(i))
    Y_test = np.load('data/ten_fold_data/Y_valid_{}.npy'.format(i))
    
    X_train = np.append(X_train,X_semi , axis=0)
    Y_train = np.append(Y_train,Y_semi , axis=0)
    
    X_train , Y_train = shuffle(X_train, Y_train, random_state=5)
    
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    
    model = load_model(join(model_path,'best_{}.h5'.format(i)))
    
    checkpoint = ModelCheckpoint(join(refine_path , 'semi_self_%d_{val_acc:.5f}.h5'%i), monitor='val_acc', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_acc", mode="max", patience=30)
    callbacks_list = [checkpoint, early]
    
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        preprocessing_function=get_random_eraser(v_l=np.min(all_x), v_h=np.max(all_x)) # Trainset's boundaries.
    )
    
    mygenerator = MixupGenerator(X_train, Y_train, alpha=1.0, batch_size=128, datagen=datagen)
    
    model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0001),
             metrics=['accuracy'])
    # mixup
    history = model.fit_generator(mygenerator(),
                    steps_per_epoch= X_train.shape[0] // 128,
                    epochs=10000,
                    validation_data=(X_test,Y_test), callbacks=callbacks_list)
    # normalize
#     history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), callbacks=callbacks_list,
#                         batch_size=32, epochs=10000)

    
#     break

(8162, 40, 345, 1)
(8162, 41)
(371, 40, 345, 1)
(371, 41)
Epoch 1/10000


/home/mike/Desktop/venv/lib/python3.5/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


63/63 [==============================] - 26s - loss: 1.1897 - acc: 0.8578 - val_loss: 0.7869 - val_acc: 0.8652
Epoch 2/10000
63/63 [==============================] - 19s - loss: 1.1716 - acc: 0.8559 - val_loss: 0.7611 - val_acc: 0.8733
Epoch 3/10000
63/63 [==============================] - 19s - loss: 1.1473 - acc: 0.8621 - val_loss: 0.7559 - val_acc: 0.8814
Epoch 4/10000
63/63 [==============================] - 19s - loss: 1.1389 - acc: 0.8673 - val_loss: 0.7714 - val_acc: 0.8787
Epoch 5/10000
63/63 [==============================] - 20s - loss: 1.1378 - acc: 0.8709 - val_loss: 0.8263 - val_acc: 0.8760
Epoch 6/10000
63/63 [==============================] - 20s - loss: 1.1355 - acc: 0.8686 - val_loss: 0.8560 - val_acc: 0.8571
Epoch 7/10000
63/63 [==============================] - 19s - loss: 1.1378 - acc: 0.8704 - val_loss: 0.8416 - val_acc: 0.8652
Epoch 8/10000
63/63 [==============================] - 19s - loss: 1.1221 - acc: 0.8748 - val_loss: 0.8100 - val_acc: 0.8814
Epoch 9/10000


63/63 [==============================] - 19s - loss: 1.1231 - acc: 0.8693 - val_loss: 0.8358 - val_acc: 0.8491
Epoch 34/10000
63/63 [==============================] - 19s - loss: 1.1076 - acc: 0.8769 - val_loss: 0.8249 - val_acc: 0.8652
Epoch 35/10000
63/63 [==============================] - 19s - loss: 1.1140 - acc: 0.8671 - val_loss: 0.8391 - val_acc: 0.8598
Epoch 36/10000
63/63 [==============================] - 19s - loss: 1.1071 - acc: 0.8803 - val_loss: 0.7889 - val_acc: 0.8868
Epoch 37/10000
63/63 [==============================] - 20s - loss: 1.1089 - acc: 0.8690 - val_loss: 0.7828 - val_acc: 0.8868
Epoch 38/10000
63/63 [==============================] - 21s - loss: 1.1085 - acc: 0.8775 - val_loss: 0.8892 - val_acc: 0.8491
Epoch 39/10000
63/63 [==============================] - 20s - loss: 1.1034 - acc: 0.8760 - val_loss: 0.8126 - val_acc: 0.8706
Epoch 40/10000
63/63 [==============================] - 21s - loss: 1.1065 - acc: 0.8735 - val_loss: 0.7793 - val_acc: 0.8679
Epoch 4

63/63 [==============================] - 19s - loss: 1.0928 - acc: 0.8796 - val_loss: 0.7345 - val_acc: 0.8814
Epoch 67/10000
63/63 [==============================] - 19s - loss: 1.0998 - acc: 0.8769 - val_loss: 0.7549 - val_acc: 0.8895
Epoch 68/10000
63/63 [==============================] - 19s - loss: 1.0802 - acc: 0.8751 - val_loss: 0.8015 - val_acc: 0.8706
Epoch 69/10000
63/63 [==============================] - 19s - loss: 1.0848 - acc: 0.8771 - val_loss: 0.7860 - val_acc: 0.8760
Epoch 70/10000
63/63 [==============================] - 19s - loss: 1.0952 - acc: 0.8759 - val_loss: 0.7203 - val_acc: 0.8868
Epoch 71/10000
63/63 [==============================] - 19s - loss: 1.0841 - acc: 0.8733 - val_loss: 0.7296 - val_acc: 0.8895
Epoch 72/10000
63/63 [==============================] - 19s - loss: 1.0752 - acc: 0.8836 - val_loss: 0.7399 - val_acc: 0.8949
Epoch 73/10000
63/63 [==============================] - 19s - loss: 1.0891 - acc: 0.8837 - val_loss: 0.8104 - val_acc: 0.8598
(8162, 

63/63 [==============================] - 20s - loss: 1.0174 - acc: 0.8822 - val_loss: 0.7042 - val_acc: 0.8814
Epoch 26/10000
63/63 [==============================] - 19s - loss: 1.0118 - acc: 0.8880 - val_loss: 0.7108 - val_acc: 0.8760
Epoch 27/10000
63/63 [==============================] - 19s - loss: 1.0055 - acc: 0.8890 - val_loss: 0.6980 - val_acc: 0.8760
Epoch 28/10000
63/63 [==============================] - 19s - loss: 1.0060 - acc: 0.8876 - val_loss: 0.7385 - val_acc: 0.8652
Epoch 29/10000
63/63 [==============================] - 19s - loss: 1.0099 - acc: 0.8842 - val_loss: 0.6976 - val_acc: 0.8787
Epoch 30/10000
63/63 [==============================] - 19s - loss: 1.0066 - acc: 0.8903 - val_loss: 0.7622 - val_acc: 0.8733
Epoch 31/10000
63/63 [==============================] - 19s - loss: 1.0060 - acc: 0.8916 - val_loss: 0.7078 - val_acc: 0.8814
Epoch 32/10000
63/63 [==============================] - 19s - loss: 1.0053 - acc: 0.8875 - val_loss: 0.7123 - val_acc: 0.8976
Epoch 3

63/63 [==============================] - 19s - loss: 1.0084 - acc: 0.8844 - val_loss: 0.7185 - val_acc: 0.8733
Epoch 59/10000
63/63 [==============================] - 19s - loss: 1.0082 - acc: 0.8829 - val_loss: 0.7568 - val_acc: 0.8598
Epoch 60/10000
63/63 [==============================] - 19s - loss: 1.0032 - acc: 0.8828 - val_loss: 0.7364 - val_acc: 0.8760
Epoch 61/10000
63/63 [==============================] - 19s - loss: 0.9970 - acc: 0.8925 - val_loss: 0.7048 - val_acc: 0.8760
Epoch 62/10000
63/63 [==============================] - 19s - loss: 1.0011 - acc: 0.8872 - val_loss: 0.6912 - val_acc: 0.8814
Epoch 63/10000
63/63 [==============================] - 21s - loss: 0.9950 - acc: 0.8879 - val_loss: 0.6963 - val_acc: 0.8733
(8162, 40, 345, 1)
(8162, 41)
(371, 40, 345, 1)
(371, 41)


KeyboardInterrupt: 